In [1]:
import os
import random
import sys
 
if len(sys.argv) < 2:
    print("no directory specified, please input target directory")
    exit()
 
root_path = sys.argv[1]
 
xmlfilepath = 'D:/workflow/keras-yolo3-master/VOCdevkit/VOC2007/Annotations'
txtsavepath = 'D:/workflow/keras-yolo3-master/VOCdevkit/VOC2007/ImageSets/Main'
 
if not os.path.exists(root_path):
    print("cannot find such directory: " + root_path)
    exit()
 
if not os.path.exists(txtsavepath):
    os.makedirs(txtsavepath)
 
trainval_percent = 0.9
train_percent = 0.8
total_xml = os.listdir(xmlfilepath)
num = len(total_xml)
list = range(num)
tv = int(num * trainval_percent)
tr = int(tv * train_percent)
trainval = random.sample(list, tv)
train = random.sample(trainval, tr)
 
print("train and val size:", tv)
print("train size:", tr)
 
ftrainval = open(txtsavepath + '/trainval.txt', 'w')
ftest = open(txtsavepath + '/test.txt', 'w')
ftrain = open(txtsavepath + '/train.txt', 'w')
fval = open(txtsavepath + '/val.txt', 'w')
 
for i in list:
    name = total_xml[i][:-4] + '\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)
 
ftrainval.close()
ftrain.close()
fval.close()
ftest.close()

train and val size: 89
train size: 71


In [2]:
   annotation_path = '2007_train.txt'
   log_dir = 'logs/000/'
   classes_path = 'model_data/voc_classes.txt'
   anchors_path = 'model_data/yolo_anchors.txt'

In [3]:
import tensorflow as tf
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data
def _main(): 
    annotation_path = '2007_train.txt'
    log_dir = 'logs/000/'
    classes_path = 'model_data/voc_classes.txt'
    anchors_path = 'model_data/yolo_anchors.txt'
    class_names = get_classes(classes_path)
    anchors = get_anchors(anchors_path)
    input_shape = (416,416) # multiple of 32, hw
    model = create_model(input_shape, anchors, len(class_names) )
    train(model, annotation_path, input_shape, anchors, len(class_names), log_dir=log_dir)   
    sess.close()
def train(model, annotation_path, input_shape, anchors, num_classes, log_dir='logs/'):
    model.compile(optimizer='adam', loss={
        'yolo_loss': lambda y_true, y_pred: y_pred})
    logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + "ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5",
        monitor='val_loss', save_weights_only=True, save_best_only=True, period=1)
    batch_size = 2
    val_split = 0.1
    with open(annotation_path) as f:
        lines = f.readlines()
    np.random.shuffle(lines)
    num_val = int(len(lines)*val_split)
    num_train = len(lines) - num_val
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    model.fit_generator(data_generator_wrap(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrap(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=200,
            initial_epoch=0)
    model.save_weights(log_dir + 'trained_weights1-1.h5')
def get_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names
def get_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)
def create_model(input_shape, anchors, num_classes, load_pretrained=False, freeze_body=False,
            weights_path='model_data/yolo_weights.h5'):
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)
    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]
    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))
    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body:
            # Do not freeze 3 output layers.
            num = len(model_body.layers)-7
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))
    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)
    return model
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    np.random.shuffle(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            i %= n
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i += 1
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)
def data_generator_wrap(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)
if __name__ == '__main__':    
    _main()


Using TensorFlow backend.


Create YOLOv3 model with 9 anchors and 1 classes.
Train on 64 samples, val on 7 samples, with batch size 2.
Epoch 1/200
32/32 [==============================] - 43s 1s/step - loss: 2243.6259 - val_loss: 111958319.3438
Epoch 2/200
32/32 [==============================] - 22s 688ms/step - loss: 284.2927 - val_loss: 78788.8672
Epoch 3/200
32/32 [==============================] - 22s 688ms/step - loss: 180.0506 - val_loss: 362194.9848
Epoch 4/200
32/32 [==============================] - 22s 689ms/step - loss: 162.8722 - val_loss: 281.9111
Epoch 5/200
32/32 [==============================] - 22s 689ms/step - loss: 118.2100 - val_loss: 344.8392
Epoch 6/200
32/32 [==============================] - 22s 688ms/step - loss: 96.4972 - val_loss: 115.6825
Epoch 7/200
32/32 [==============================] - 22s 689ms/step - loss: 83.9033 - val_loss: 92.9861
Epoch 8/200
32/32 [==============================] - 22s 689ms/step - loss: 77.9453 - val_loss: 81.6338
Epoch 9/200
32/32 [=====================

32/32 [==============================] - 22s 691ms/step - loss: 52.6116 - val_loss: 53.5324
Epoch 155/200
32/32 [==============================] - 22s 691ms/step - loss: 52.7767 - val_loss: 54.5442
Epoch 156/200
32/32 [==============================] - 22s 691ms/step - loss: 49.9940 - val_loss: 47.1934
Epoch 157/200
32/32 [==============================] - 22s 690ms/step - loss: 52.1944 - val_loss: 59.0074
Epoch 158/200
32/32 [==============================] - 22s 690ms/step - loss: 50.1699 - val_loss: 53.4573
Epoch 159/200
32/32 [==============================] - 22s 690ms/step - loss: 48.7486 - val_loss: 56.9240
Epoch 160/200
32/32 [==============================] - 22s 689ms/step - loss: 50.4433 - val_loss: 54.3899
Epoch 161/200
32/32 [==============================] - 22s 690ms/step - loss: 49.4565 - val_loss: 51.6772
Epoch 162/200
32/32 [==============================] - 22s 692ms/step - loss: 49.2253 - val_loss: 51.7621
Epoch 163/200
32/32 [==============================] - 22s 6

NameError: name 'sess' is not defined

In [1]:
# -*- coding: utf-8 -*-
"""
Class definition of YOLO_v3 style detection model on image and video
"""
import colorsys
import os
import sys 
from timeit import default_timer as timer
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw
from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
from keras.utils import multi_gpu_model
class YOLO(object):
    _defaults = {
        "model_path": 'logs/000/trained_weights1.h5', ##训练好的模型的路径
        "anchors_path": 'model_data/yolo_anchors.txt',
        "classes_path": 'model_data/voc_classes.txt',
        "score" : 0.3,
        "iou" : 0.45,
        "model_image_size" : (416, 416),
        "gpu_num" : 0
    }
    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"
    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()
    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names
    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)
    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'
        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'
        print('{} model, anchors, and classes loaded.'.format(model_path))
        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.
        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes
    def detect_image(self, image):
        start = timer()
        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')
        print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })
        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))
        font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300
        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]
            label = '{} {:.2f}'.format(predicted_class, score)
            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)
            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            print(label, (left, top), (right, bottom))
            image2 = []
            cellname = timer()
            cellpath = './result/'
            image2 = image.crop((left,top,right,bottom))
            image2.save(cellpath+str(timer())+'.jpg')
            #new_f=open("/home/shan/xws/pro/keras-yolo3/detection-results/"+tmp_file.replace(".jpg", ".txt"), "a")
            new_f.write("%s %s %s %s %s\n" %  (label, left, top, right, bottom))
            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])
            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                draw.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=self.colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=self.colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw
        end = timer()
        print(end - start)
        return image
    def close_session(self):
        self.sess.close()
        
if __name__ == '__main__':
    # yolo=YOLO()
    # path = '1.jpg'
    # try:
    #     image = Image.open(path)
    # except:
    #     print('Open Error! Try again!')
    # else:
    #     r_image = yolo.detect_image(image)
    #     r_image.show()
    # yolo.close_session()
    #strat1=timer()
    dirname='./test/'  ##该目录为测试照片的存储路径，每次测试照片的数量可以自己设定
    path=os.path.join(dirname)
    pic_list=os.listdir(path) 
    count=0
    yolo=YOLO()
    for filename in pic_list:
        tmp_file=pic_list[count]
        new_f=open('./result'+tmp_file.replace(".jpg", ".txt"), "a")  #预测坐标生成txt文件保存的路径
        abs_path=path+pic_list[count]
        image = Image.open(abs_path)
        r_image = yolo.detect_image(image)
        count=count+1
        r_image.show()
    #end1=timer()
    print(count)
    yolo.close_session()

Using TensorFlow backend.


logs/000/trained_weights1.h5 model, anchors, and classes loaded.
(416, 416, 3)
Found 1 boxes for img
cell 0.40 (305, 75) (416, 201)
2.6282370000000004
(416, 416, 3)
Found 2 boxes for img
cell 0.37 (69, 313) (212, 416)
cell 0.48 (303, 4) (416, 93)
0.14089859999999987
(416, 416, 3)
Found 2 boxes for img
cell 0.35 (69, 312) (212, 416)
cell 0.44 (302, 9) (416, 87)
0.11178509999999964
(416, 416, 3)
Found 1 boxes for img
cell 0.39 (308, 105) (416, 230)
0.10923170000000049
(416, 416, 3)
Found 1 boxes for img
cell 0.45 (320, 311) (407, 416)
0.11002590000000012
(416, 416, 3)
Found 0 boxes for img
0.1071049999999989
(416, 416, 3)
Found 3 boxes for img
cell 0.36 (323, 251) (400, 416)
cell 0.47 (170, 0) (298, 94)
cell 0.48 (0, 0) (159, 84)
0.11252599999999902
(416, 416, 3)
Found 1 boxes for img
cell 0.36 (204, 320) (330, 406)
0.11090979999999995
(416, 416, 3)
Found 1 boxes for img
cell 0.39 (172, 216) (296, 340)
0.1098504000000009
(416, 416, 3)
Found 0 boxes for img
0.1074780999999998
(416, 416, 3

KeyboardInterrupt: 